In [62]:
from zget_data import get_responses, get_qdetails, get_questions, page_map, quest_map, attention
from collections import defaultdict
import pandas as pd
import json, os
from selenium import webdriver
from selenium.webdriver.common.by import By

In [63]:
password_response, time_response, survey_response, dids = get_responses()

Total: 163; 
Attentive: 131; Non Attentive: 32


In [64]:
password = pd.DataFrame(password_response)
passwords = password['password2'].to_list()

pass_strength = {}

pfile = "data/password/pstrength.json"
if os.path.exists(pfile):
    with open(pfile, "r") as fp:
        pass_strength.update(json.load(fp))
else:
    driver = webdriver.Chrome()
    driver.get("https://passwordmonster.com")

    for pss in passwords:
        element = driver.find_element(By.ID, 'lgd_out_pg_pass')
        element.send_keys(pss)
        element.clear()
        complexity = driver.find_element(By.ID, 'complexity-span')
        pass_strength[pss] = complexity.get_attribute("innerHTML")

    with open("password/pstrength.json", "w") as fp:
        json.dump(pass_strength, fp)

In [65]:
pass_stmap = {
    'Very Weak': -2,
    'Weak': -1,
    'Medium': 0,
    'Strong': 1,
    'Very Strong': 2,
}
password['actual_strength'] = password['password2'].apply(lambda x: pass_stmap.get(pass_strength.get(x)))
password

,password2,password2_recall1,password2_recall2,password2_recall3,actual_strength
0,Vistech@2022,NaN,NaN,NaN,1
1,B1E9B6E44691,b1e9b6e44691,NaN,NaN,2
2,Zydynamix414!,NaN,NaN,NaN,2
3,moasdjkleR-,NaN,NaN,NaN,2
4,al32QW!,NaN,NaN,NaN,0
...,...,...,...,...,...
126,Zarta$$750,NaN,NaN,NaN,1
127,1&sak56Uk65i,NaN,NaN,NaN,2
128,a#p2022,NaN,NaN,NaN,0
129,Cookiecrumbles2!,NaN,NaN,NaN,0


In [66]:
survey_response_df = pd.DataFrame(survey_response)

page_id = '3'
normqsid = ['1-1', '1-4']
fpass = pd.DataFrame(survey_response_df[page_id].to_dict()).astype(int)
normfpass = 8 - fpass.loc[normqsid]
fpass.update(normfpass)

# normalize scale 1 to 8 to scale -3 to 3
# normfpass = fpass - 4
# fpass.update(normfpass)

new_index = {}
for qsid in fpass.index:
    likert_scale = 'low' if qsid in normqsid else 'high'
    new_index[qsid] = json.loads(get_qdetails(page_id, qsid).replace("'", '"').split('\n')[1]).get('title')
fpass.rename(columns=new_index, inplace=True)
fpass = fpass.rename(new_index)
pass_behaviour = pd.DataFrame(fpass.T.mean(axis=1), columns=['Password Behaviour'])
pass_behaviour

,Password Behaviour
0,5.50
1,7.00
2,5.25
3,6.50
4,4.00
...,...
126,6.00
127,7.00
128,3.25
129,5.00


In [67]:
page_id = '12'
normqsid = []
spass = pd.DataFrame(survey_response_df[page_id].to_dict()).astype(int)
spass.drop(attention.get('12'), inplace=True)
normspass = 8 - spass.loc[normqsid]
spass.update(normspass)

# normalize scale 1 to 8 to scale -3 to 3
# normspass = spass - 4
# spass.update(normspass)

new_index = {}
for qsid in spass.index:
    new_index[qsid] = json.loads(get_qdetails(page_id, qsid).replace("'", '"').split('\n')[1]).get('title')
new_index
spass = spass.rename(new_index)
spass = spass.T
spass

,strength of password,motivated after going through design,Hard to remember,Can be easily guessed by a hacker,Can be easily guessed by my close friends or family members,Stronger than the passwords I usually use
0,5,5,4,5,4,5
1,6,7,4,2,1,2
2,6,7,2,2,1,2
3,4,4,5,3,1,1
4,3,5,6,4,3,3
...,...,...,...,...,...,...
126,7,7,1,1,1,4
127,7,7,7,1,1,4
128,6,5,6,6,5,4
129,6,7,2,1,1,4
